In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/nlp_ss24/multilingual-lexical-simplification')

In [3]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/nlp_ss24/multilingual-lexical-simplification']

In [5]:
from src.lexical_simplifier import LexicalSimplifier

ModuleNotFoundError: No module named 'src'

In [ ]:
import abc
from typing import List
from abc import ABCMeta


class LexicalSimplifier(metaclass=ABCMeta):
    """
    An abstract base class for a lexical simplification. Implements a basic initializer and functions for
    setting the model and pattern for the simplifier. The main function to be implemented by subclasses is
    get_substitutions_for, which should return a list of possible substitutions for a given complex word.

    Attributes:
        model: The model used for generating substitutions.
        pattern: The format string used to dynamically build prompts for generating substitutions. In the simplest
            case this could be a string with one placeholder, into which the input sentence with the masked complex
            word is inserted (eg. for a BERT model).
        exemplars: A list of exemplar words used for zero-shot learning. If provided, these will be prepended to the every
            prompt.
    """
    model = None
    pattern: str = None
    exemplars: List[str] = None

    def __init__(self, model, pattern, exemplars):
        if model is None:
            raise ValueError("Please initialize the model for this LexicalSimplifier.")

        if pattern is None or pattern == "":
            raise ValueError("Please provide a pattern for this LexicalSimplifier.\n"
                             "A pattern should be a format string that can take at least one argument.")

        if exemplars is None or len(exemplars) == 0:
            raise UserWarning("No exemplars provided, using zero-shot mode.")

        self.model = model
        self.pattern = pattern
        self.exemplars = exemplars

    def set_model(self, model):
        self.model = model

    def set_pattern(self, pattern):
        self.pattern = pattern

    @abc.abstractmethod
    def generate_substitutions_for(self, complex_word) -> List[str]:
        """Generates a list of substitutions via the model predictions for the given complex word."""
        raise NotImplementedError("Please implement this method in the subclass.")

In [ ]:
from lexical_simplifier import LexicalSimplifier


class GermanBertLexicalSimplifier(LexicalSimplifier):
    """
    A German BERT based implementation of lexical simplification. Masks the given complex word with [MASK], adds other
    BERT specific tokens and generates a list of possible substitutions via the model predictions based on the prompt.
    """

    def __init__(self, model, pattern, exemplars):
        super().__init__(model, pattern, exemplars)

    def generate_substitutions_for(self, complex_word):
        """Generates a list of substitutions via the model predictions for the given complex word."""
        raise NotImplementedError("Please implement this method in the subclass.")